In [ ]:
!pip install fcapy[all]
!pip install frozendict
!pip install ipynb
!pip install sparselinear
!pip install bitsets
!pip install bitarray
!pip install torch-scatter -f https://data.pyg.org/whl/torch-2.0.0+cuda118.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-2.0.0+cuda118.html
!pip install torch-cluster -f https://data.pyg.org/whl/torch-2.0.0+cuda118.html
!pip install git+https://github.com/pyg-team/pytorch_geometric.git

Looking in links: https://data.pyg.org/whl/torch-2.0.0+cuda118.html
Looking in links: https://data.pyg.org/whl/torch-2.0.0+cuda118.html
Looking in links: https://data.pyg.org/whl/torch-2.0.0+cuda118.html
  Cloning https://github.com/pyg-team/pytorch_geometric.git to /tmp/pip-req-build-9_802_vy
  Running command git clone --filter=blob:none --quiet https://github.com/pyg-team/pytorch_geometric.git /tmp/pip-req-build-9_802_vy
  Resolved https://github.com/pyg-team/pytorch_geometric.git to commit da1ed7f77cee21f3312ee9be6930e6a46f473781
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
from fcapy.context import FormalContext
from fcapy.lattice import ConceptLattice
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, recall_score, accuracy_score, classification_report
import torch

In [ ]:
from fcapy.visualizer import LineVizNx
import matplotlib.pyplot as plt

plt.rcParams['figure.facecolor'] = (1,1,1,1)

In [ ]:
import neural_lib as nl

### Dataset: Job Applicants Data (Human Resource)


In [1]:
import kagglehub
import pandas as pd
import numpy as np

path = kagglehub.dataset_download("ayushtankha/70k-job-applicants-data-human-resource")
print("Path to dataset files:", path)
data_path = f"{path}/stackoverflow_full.csv"
df1 = pd.read_csv(data_path)
df1.drop(columns='Unnamed: 0', inplace=True) #preprocessing
df1.drop_duplicates(inplace=True)
df1.dropna(inplace=True)
df1 = df1.head(1000) # 75к observations is too much and long for model's training
df1

100%|██████████| 2.02M/2.02M [00:00<00:00, 2.48MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/ayushtankha/70k-job-applicants-data-human-resource/versions/1


,Age,Accessibility,EdLevel,Employment,Gender,MentalHealth,MainBranch,YearsCode,YearsCodePro,Country,PreviousSalary,HaveWorkedWith,ComputerSkills,Employed
0,<35,No,Master,1,Man,No,Dev,7,4,Sweden,51552.0,C++;Python;Git;PostgreSQL,4,0
1,<35,No,Undergraduate,1,Man,No,Dev,12,5,Spain,46482.0,Bash/Shell;HTML/CSS;JavaScript;Node.js;SQL;Typ...,12,1
2,<35,No,Master,1,Man,No,Dev,15,6,Germany,77290.0,C;C++;Java;Perl;Ruby;Git;Ruby on Rails,7,0
3,<35,No,Undergraduate,1,Man,No,Dev,9,6,Canada,46135.0,Bash/Shell;HTML/CSS;JavaScript;PHP;Ruby;SQL;Gi...,13,0
4,>35,No,PhD,0,Man,No,NotDev,40,30,Singapore,160932.0,C++;Python,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,<35,No,Undergraduate,1,Man,No,Dev,6,2,Egypt,11544.0,Python;Docker;Git,3,0
996,<35,No,Master,1,Man,No,Dev,9,5,Romania,35280.0,Bash/Shell;C#;PowerShell;Python;SQL;Git;ASP.NE...,12,1
997,>35,No,Master,0,Man,No,Dev,20,15,Portugal,95126.0,Assembly;C;C#;C++;HTML/CSS;JavaScript;Matlab;P...,25,1
998,>35,No,PhD,1,Man,No,Dev,25,15,France,81074.0,Bash/Shell;Elixir;Python;SQL;Ansible;Docker;Gi...,12,0


In [ ]:
df1['Employed'].value_counts() #training sample are balanced

0    503
1    497
Name: Employed, dtype: int64

In [3]:
df = df1.drop(columns=['HaveWorkedWith']) # here will be about 70к+ features, if we use One-Hot Encoding

unique_countries = df1['Country'].nunique()  # Count of unique countries
print(f"Number of unique countries in the first 1000 rows: {unique_countries}") #90 feutures based on countries is hard to interpret

df = df.drop(columns=['Country']) #drop this feature

Number of unique countries in the first 1000 rows: 90


In [4]:
# One-Hot Encoding for binarization of categorical features
df_encoded = pd.get_dummies(df, drop_first=True)


# calculates the threshold for numeric features ​​(average)
numerical_cols = ['Employment', 'YearsCode', 'YearsCodePro', 'PreviousSalary', 'ComputerSkills']
threshold = df[numerical_cols].mean()

for col in numerical_cols:
    df_encoded[col + '_bin'] = (df[col] > threshold[col])

# 0 и 1 -> True и False
df_encoded = df_encoded.astype(bool)

df_encoded.reset_index(drop=True, inplace=True)
df_encoded.index = 'employ' + (df_encoded.index + 1).astype(str)
df_encoded.to_csv('job_bin.csv')
df_encoded

,Employment,YearsCode,YearsCodePro,PreviousSalary,ComputerSkills,Employed,Age_>35,Accessibility_Yes,EdLevel_NoHigherEd,EdLevel_Other,...,EdLevel_Undergraduate,Gender_NonBinary,Gender_Woman,MentalHealth_Yes,MainBranch_NotDev,Employment_bin,YearsCode_bin,YearsCodePro_bin,PreviousSalary_bin,ComputerSkills_bin
employ1,True,True,True,True,True,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
employ2,True,True,True,True,True,True,False,False,False,False,...,True,False,False,False,False,True,False,False,False,False
employ3,True,True,True,True,True,False,False,False,False,False,...,False,False,False,False,False,True,True,False,True,False
employ4,True,True,True,True,True,False,False,False,False,False,...,True,False,False,False,False,True,False,False,False,True
employ5,False,True,True,True,True,False,True,False,False,False,...,False,False,False,False,True,False,True,True,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
employ996,True,True,True,True,True,False,False,False,False,False,...,True,False,False,False,False,True,False,False,False,False
employ997,True,True,True,True,True,True,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
employ998,False,True,True,True,True,True,True,False,False,False,...,False,False,False,False,False,False,True,True,True,True
employ999,True,True,True,True,True,False,True,False,False,False,...,False,False,False,False,False,True,True,True,True,False


In [6]:
y_feat = 'Employed'
df_train, df_test = train_test_split(df_encoded, train_size=0.7, random_state=0)
X_train, y_train = df_train.drop(y_feat, axis=1), df_train[y_feat]
X_test, y_test = df_test.drop(y_feat, axis=1), df_test[y_feat]
X_train.describe()

,Employment,YearsCode,YearsCodePro,PreviousSalary,ComputerSkills,Age_>35,Accessibility_Yes,EdLevel_NoHigherEd,EdLevel_Other,EdLevel_PhD,EdLevel_Undergraduate,Gender_NonBinary,Gender_Woman,MentalHealth_Yes,MainBranch_NotDev,Employment_bin,YearsCode_bin,YearsCodePro_bin,PreviousSalary_bin,ComputerSkills_bin
count,700,700,700,700,700,700,700,700,700,700,700,700,700,700,700,700,700,700,700,700
unique,2,1,2,1,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
top,True,True,True,True,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False
freq,604,700,677,700,700,490,683,668,591,672,373,691,678,571,627,604,422,450,419,377


In [ ]:
X_test.describe()

,Employment,YearsCode,YearsCodePro,PreviousSalary,ComputerSkills,Age_>35,Accessibility_Yes,EdLevel_NoHigherEd,EdLevel_Other,EdLevel_PhD,EdLevel_Undergraduate,Gender_NonBinary,Gender_Woman,MentalHealth_Yes,MainBranch_NotDev,Employment_bin,YearsCode_bin,YearsCodePro_bin,PreviousSalary_bin,ComputerSkills_bin
count,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300
unique,2,1,2,1,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
top,True,True,True,True,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,True
freq,263,300,291,300,300,197,288,289,258,283,151,296,290,244,270,263,172,172,168,151


In [ ]:
# Create a Formal Context from the training data
K_train = FormalContext.from_pandas(X_train)

# Print the Formal Context
K_train

FormalContext (700 objects, 20 attributes, 6073 connections)
         |Employment|YearsCode|YearsCodePro|PreviousSalary|ComputerSkills|...|Employment_bin|YearsCode_bin|YearsCodePro_bin|PreviousSalary_bin|ComputerSkills_bin|
employ106|         X|        X|           X|             X|             X|...|             X|             |                |                 X|                  |
employ69 |         X|        X|           X|             X|             X|...|             X|             |                |                  |                 X|
employ480|         X|        X|           X|             X|             X|...|             X|            X|               X|                 X|                  |
employ400|         X|        X|           X|             X|             X|...|             X|            X|                |                 X|                 X|
employ435|         X|        X|           X|             X|             X|...|             X|             |                |

In [ ]:
L = ConceptLattice.from_context(K_train, algo = 'Sofia', is_monotone=True)
len(L) #the number of concepts in the lattice

98

In [ ]:
for c in L:
    y_preds = np.zeros(K_train.n_objects)
    y_preds[list(c.extent_i)] = 1
    c.measures['f1_score'] = f1_score(y_train, y_preds)
    c.measures['recall_score'] = recall_score(y_train, y_preds)

In [ ]:
concepts_count = 50 #took 50 best concepts
best_concepts = list(L.measures['f1_score'].argsort()[::-1][:concepts_count])

assert len({g_i for c in L[best_concepts] for g_i in c.extent_i})==K_train.n_objects, "Selected concepts do not cover all train objects"

In [ ]:
cn = nl.ConceptNetwork.from_lattice(L, best_concepts, sorted(set(y_train)))

In [ ]:
vis = LineVizNx(
    node_label_font_size=14,
    node_label_func=lambda el_i, P: nl.neuron_label_func(el_i, P, set(cn.attributes))+'\n\n',
    edge_cmap="hsv"
    )
set(X_train.columns)

{'Accessibility_Yes',
 'Age_>35',
 'ComputerSkills',
 'ComputerSkills_bin',
 'EdLevel_NoHigherEd',
 'EdLevel_Other',
 'EdLevel_PhD',
 'EdLevel_Undergraduate',
 'Employment',
 'Employment_bin',
 'Gender_NonBinary',
 'Gender_Woman',
 'MainBranch_NotDev',
 'MentalHealth_Yes',
 'PreviousSalary',
 'PreviousSalary_bin',
 'YearsCode',
 'YearsCodePro',
 'YearsCodePro_bin',
 'YearsCode_bin'}

In [ ]:
fig, ax = plt.subplots(figsize=(50,30))

vis.draw_poset(
    cn.poset, ax=ax, edge_color='grey',
    flg_node_indices=False,
    node_label_func=lambda el_i, P: nl.neuron_label_func(el_i, P, set(cn.attributes), only_new_attrs=True)+'\n\n',
    node_color='black',
)

plt.title(f'NN based on {concepts_count} best concepts from monotone concept lattice', loc='center', size=24)

plt.subplots_adjust()
plt.tight_layout()
plt.savefig('job.png')
plt.show()

In [ ]:
import torch
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
cn = nl.ConceptNetwork.from_lattice(L, best_concepts, sorted(set(y_train)))
cn.fit(X_train, y_train,  n_epochs = 2500, nonlinearity=torch.nn.GELU)

In [ ]:
print('Class predictions \n', cn.predict(X_test).numpy().astype(bool))
print('True classes \n', y_test.values)

Class predictions 
 [ True  True False False  True False False  True  True False False  True
 False False  True  True False False False  True False False False  True
 False  True  True  True  True False  True False False  True False False
 False False False False  True False False  True  True  True False  True
 False False False False False False  True  True  True False  True  True
  True False  True False False  True False False False  True  True False
  True False False  True  True  True False False False  True  True False
  True False  True  True False False  True  True  True  True False False
 False  True  True False False False  True  True  True False  True  True
  True  True  True False False False False  True False False  True  True
  True False False False  True False False  True False False  True  True
 False False False False False False False  True False  True  True  True
  True False False  True  True  True False  True False False  True False
  True  True False  True False 

In [ ]:
print("F1 score =", f1_score(y_test.values, cn.predict(X_test).numpy()))

F1 score = 0.8239202657807307


In [ ]:
y_pred = cn.predict(X_test).numpy()
y_proba = cn.predict_proba(X_test).detach().numpy()

print('Recall score:', recall_score(y_test.values.astype('int'), y_pred))
print('F1 score:', f1_score(y_test.values.astype('int'), y_pred))
print('Accuracy score:', accuracy_score(y_test.values.astype('int'), y_pred))

Recall score: 0.8104575163398693
F1 score: 0.8239202657807307
Accuracy score: 0.8233333333333334


In [ ]:
edge_weights = cn.edge_weights_from_network()

In [ ]:
import networkx as nx
fig, ax = plt.subplots(figsize=(50,30))

vis.draw_poset(
    cn.poset, ax=ax, node_color='black',
    flg_node_indices=False,
    node_label_func=lambda el_i, P: nl.neuron_label_func(el_i, P, set(cn.attributes), only_new_attrs=True)+'\n\n',
    edge_color=[edge_weights[edge] for edge in cn.poset.to_networkx().edges],
    edge_cmap=plt.cm.RdBu,
)
nx.draw_networkx_edge_labels(cn.poset.to_networkx(), vis.mover.pos, {k: f"{v:.1f}" for k,v in edge_weights.items()}, label_pos=0.7)

plt.title('Neural network with fitted edge weights', size=24, loc='center')
plt.tight_layout()
plt.subplots_adjust()
plt.savefig('job_fitted.png')
plt.show()